# Offline Policy Evaluation w/ minibatch

Testing offline policy evalation with zone-agnostic behavior policy model

In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import pandas as pd
import pickle
sys.path.insert(0, "../")
import torch
from ppo import PPO
import glob
from tqdm.notebook import tqdm
import json
from scipy.stats import spearmanr
import importlib
import traceback

%matplotlib widget

In [2]:
# log_data = pd.read_csv("../data/rule_based_log_data/15zone/0_cleaned_log.csv")
# with open("../data/rule_based_log_data/15zone/action_probs_all_data.pkl", "rb") as f:
#     behavior_model = pickle.load(f)
# # with open("../data/invalid_policy_list_20220705.json") as f:
# #     invalid_policies = json.load(f)["invalid_policies"]
# invalid_policies = []

log_data = pd.read_csv("../data/rule_based_log_data/15zone/0_cleaned_log.csv")
with open("../data/rule_based_log_data/15zone/action_probs_all_data.pkl", "rb") as f:
    behavior_model = pickle.load(f)
# with open("../data/invalid_policy_list_20220705.json") as f:
#     invalid_policies = json.load(f)["invalid_policies"]
invalid_policies = []

## 30 Day MiniBatch

In [3]:
num_ts_per_day = 4 * 24
num_days = 15
ts_end = num_ts_per_day * num_days
zones = log_data["zone"].unique()

## Loading all Policies

In [4]:
policy_list = sorted(list(glob.glob(f"../policy_library_20220820/**.pth")))

### 1. Inverse Probability Weighting

In [6]:
from ope.iw import InverseProbabilityWeighting
policy_scores = {}
for zone in zones:
    print(zone)
    ope_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
    ope_data = log_data[:ts_end]
    ipw = InverseProbabilityWeighting(ope_data, retain_grad_fn=False, univariate_action=True)
    for policy in tqdm(policy_list):
        agent = PPO(6, 1, 0.003, 0.0005, 1, 10, 0.2,
                    has_continuous_action_space=True, action_std_init=0.2, 
                    device=torch.device('cpu'), diverse_policies=list(),
                    diverse_weight=0, diverse_increase=True)
        agent.load(policy)
        agent.policy_evaluation = True
        agent.policy_old.set_action_std(0.1)
        if policy not in invalid_policies:
            score, _, _, _, _ = ipw.evaluate_policy(agent.select_action, behavior_model, score="mean")
        else:
            continue
        if policy not in policy_scores:
            policy_scores[policy] = {}
        if zone not in policy_scores[policy]:
            policy_scores[policy][zone] = score.item()

Perimeter_top_ZN_3


  0%|          | 0/870 [00:00<?, ?it/s]

Perimeter_bot_ZN_1


  0%|          | 0/870 [00:00<?, ?it/s]

Perimeter_top_ZN_1


...

  0%|          | 0/870 [00:00<?, ?it/s]

Perimeter_mid_ZN_1


  0%|          | 0/870 [00:00<?, ?it/s]

Perimeter_bot_ZN_4


  0%|          | 0/870 [00:00<?, ?it/s]

In [7]:
eval_data_loc = "../data/evaluation_report_20220820.csv"
df = pd.read_csv(eval_data_loc, header=None, names=["datetime","policy","zone","energy"])
spearman_corr = {}
for zone in zones:
    eval_df = df[df["zone"] == zone]

    # invalid_policies = list(set(policy_list) - set(policy_scores.keys()))
    for i_policy in invalid_policies:
        eval_df = eval_df[eval_df["policy"]!=i_policy[3:]]
    eval_df = eval_df.sort_values(by=["energy"])
    score_list = []
    for i, row in eval_df.iterrows():
        score_list.append(policy_scores[f"../{row['policy']}"][zone])

    eval_df["ope_scores"] = score_list
    correlation = spearmanr(eval_df["energy"].values, eval_df["ope_scores"].values)
    spearman_corr[zone] = correlation

In [8]:
spearman_corr

{'Perimeter_top_ZN_3': SpearmanrResult(correlation=-0.7156388637189263, pvalue=1.9856447169206734e-137),
 'Perimeter_bot_ZN_1': SpearmanrResult(correlation=-0.7301466779502591, pvalue=1.0142599262257645e-145),
 'Perimeter_top_ZN_1': SpearmanrResult(correlation=-0.7259029659449524, pvalue=3.0677376083860007e-143),
 'Core_bottom': SpearmanrResult(correlation=-0.7102556919609079, pvalue=1.7523344666317893e-134),
 'Perimeter_top_ZN_4': SpearmanrResult(correlation=-0.7373883772645728, pvalue=4.591473603997031e-150),
 'Core_top': SpearmanrResult(correlation=-0.7191875244846179, pvalue=2.0809227871757353e-139),
 'Perimeter_bot_ZN_2': SpearmanrResult(correlation=-0.7327660238979645, pvalue=2.826659743390074e-147),
 'Perimeter_bot_ZN_3': SpearmanrResult(correlation=-0.5451104390958015, pvalue=1.7160247164011757e-68),
 'Perimeter_mid_ZN_4': SpearmanrResult(correlation=-0.7354066952181584, pvalue=7.327849087914692e-149),
 'Core_mid': SpearmanrResult(correlation=-0.7141827342487238, pvalue=1.26312

In [11]:
with open(f"data/15zone/ipw/raw_scores/ipw_raw_scores_{num_days}_days_06_08_2022.pkl", "wb+") as f:
    pickle.dump(policy_scores, f)

In [12]:
with open(f"data/15zone/ipw/spearman_corr/ipw_spearman_corr_{num_days}_days_06_08_2022.pkl", "wb+") as f:
    pickle.dump(spearman_corr, f)

### 2. Self Normalized Inverse Probability Weighting

In [5]:
from ope.sniw import SelfNormalizedInverseProbabilityWeighting
policy_scores_sniw = {}
for zone in zones:
    print(zone)
    ope_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
    ope_data = log_data[:ts_end]
    snipw = SelfNormalizedInverseProbabilityWeighting(ope_data)
    for policy in tqdm(policy_list):
        agent = PPO(6, 1, 0.003, 0.0005, 1, 10, 0.2,
                    has_continuous_action_space=True, action_std_init=0.2, 
                    device=torch.device('cpu'), diverse_policies=list(),
                    diverse_weight=0, diverse_increase=True)
        agent.load(policy)
        agent.policy_evaluation = True
        agent.policy_old.set_action_std(0.1)
        if policy not in invalid_policies:
            score = snipw.evaluate_policy(agent.select_action, behavior_model, score="mean")
        else:
            continue
        if policy not in policy_scores_sniw:
            policy_scores_sniw[policy] = {}
        if zone not in policy_scores_sniw[policy]:
            policy_scores_sniw[policy][zone] = score.item()

Perimeter_top_ZN_3


  0%|          | 0/870 [00:00<?, ?it/s]

/Users/aakashsasikumar/Documents/MSc/Research/Dissimilar Policy/building-MARL/studies/../ope/sniw.py:60: RuntimeWarning: divide by zero encountered in log
  return np.log(value / (1- value))


Perimeter_bot_ZN_1


  0%|          | 0/870 [00:00<?, ?it/s]

...

  0%|          | 0/870 [00:00<?, ?it/s]

Perimeter_mid_ZN_1


  0%|          | 0/870 [00:00<?, ?it/s]

Perimeter_bot_ZN_4


  0%|          | 0/870 [00:00<?, ?it/s]

In [6]:
eval_data_loc = "../data/eval_data/evaluation_report_20220820.csv"
df = pd.read_csv(eval_data_loc, header=None, names=["datetime","policy","zone","energy"])
spearman_corr_sniw = {}
for zone in zones:
    eval_df = df[df["zone"] == zone]
    for i_policy in invalid_policies:
        eval_df = eval_df[eval_df["policy"]!=i_policy[3:]]
    eval_df = eval_df.sort_values(by=["energy"])
    score_list = []
    for i, row in eval_df.iterrows():
        score_list.append(policy_scores_sniw[f"../{row['policy']}"][zone])

    eval_df["ope_scores"] = score_list
    correlation = spearmanr(eval_df["energy"].values, eval_df["ope_scores"].values, nan_policy="omit")
    spearman_corr_sniw[zone] = correlation

In [7]:
with open(f"data/15zone/snipw/raw_scores/snipw_raw_scores_{num_days}_days_19_09_2022.pkl", "wb+") as f:
    pickle.dump(policy_scores_sniw, f)

In [8]:
with open(f"data/15zone/snipw/spearman_corr/snipw_spearman_corr_{num_days}_days_19_09_2022.pkl", "wb+") as f:
    pickle.dump(spearman_corr_sniw, f)

### 3. Gaussian Kernel

In [ ]:
from obp.ope import KernelizedInverseProbabilityWeighting
policy_scores_gk = {}
for zone in zones:
    print(zone)
    ope_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
    ope_data = log_data[:ts_end]
          states = []
        actions = []
        rewards = []
        for i, row in ope_data.iterrows():
            state_vars = ["outdoor_temp", "solar_irradiation", "time_hour",
                          "zone_humidity", "zone_temp", "zone_occupancy"]
            state = [row[var] for var in state_vars]
            action = row["action"]
            reward = row["reward"]
            states.append(state)
            rewards.append(reward)
            actions.append(action)
            ope = ContinuousOffPolicyEvaluation(bandit_feedback=
                                            {"action": np.array(actions),
                                             "reward": np.array(rewards),
                                             "pscore": np.ones((len(ope_data)))},
                                            ope_estimators=[KernelizedInverseProbabilityWeighting(kernel=kernel, bandwidth=bandwidth)])
    for policy in tqdm(policy_list):
        agent = PPO(6, 1, 0.003, 0.0005, 1, 10, 0.2,
                    has_continuous_action_space=True, action_std_init=0.2, 
                    device=torch.device('cpu'), diverse_policies=list(),
                    diverse_weight=0, diverse_increase=True)
        agent.load(policy)
        agent.policy_evaluation = True
        agent.policy_old.set_action_std(0.1)
        if policy not in invalid_policies:
            score = snipw.evaluate_policy(agent.select_action, behavior_model, score="mean")
        else:
            continue
        if policy not in policy_scores_sniw:
            policy_scores_sniw[policy] = {}
        if zone not in policy_scores_sniw[policy]:
            policy_scores_sniw[policy][zone] = score.item()